# http://www.bokjiro.go.kr/welInfo/retrieveLcgWelInfoList.do
## -> '지자체' 설정
# http://www.bokjiro.go.kr/welInfo/retrieveLcgWelInfoList.do?searchSidoCode=&searchCggCode=&searchIntClId=&searchCnDivCd=02&searchCtgId=&searchGb=01&searchText=&pageUnit=10&pageIndex=1


## 복지로3(지역별검색) 크롤링

## 수집 칼럼
[복지 이름, 지역구, 간단 설명, 서비스 대상, 서비스 내용, 서비스 이용 및 신청방법, 서식/자료, 문의, 사이트, 근거법령]

In [1]:
import time
import os
import json
from datetime import datetime
import math

In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from collections import defaultdict
from copy import deepcopy
from tqdm import tqdm_notebook

In [40]:
def get_total_page(url):
    """
    (string) -> int
        Args:
            url(string): 총 페이지 갯수를 알려는 page 주소
        Return:
            total_page(int): 해당 url의 총 페이지 갯수
    """
    options = webdriver.ChromeOptions()	
    options.add_argument('headless')	
    options.add_argument('disable-gpu')	
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    driver.get(url)
    total_page = int(driver.find_element_by_tag_name('#contents > div:nth-child(21) > div.catItem > div > strong').text)
    total_page = math.ceil(total_page / 10)
    
    return total_page
    

In [59]:
def clickNcrawl(url):
    """
    (string) -> list
        Args:
            url(string): Q&A 리스트
    Return:
        1) '질문'을 눌러 해당 페이지로 이동
        2) 해당 페이지에서 복지정책에 대한 정보 수집(info_dict) 및 총 복지 리스트(total_info_list)에 추가
        3) 모두 수집한 후 total_info_list 반환
    """
    
    options = webdriver.ChromeOptions()	
#     options.add_argument('headless')	
    options.add_argument('disable-gpu')	
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    driver.get(url)
    
    # 각 복지 이름에 대한 혜택을 info_dict에 담은 후
    # 모든 복지 이름에 대한 혜택을 total_info_list에 담아 리턴한다.
    total_info_list = list()
    
    for idx in range(1, 11):
        detail_page = driver.find_element_by_xpath('//*[@id="contents"]/div[4]/ul/li[%s]/div/dl/dt/a' %(str(idx)))
        detail_page.click()
        time.sleep(7)
        
        # 수집 칼럼: [복지 이름, 지역구, 간단 설명, 서비스 대상, 서비스 내용, 서비스 이용 및 신청방법, 서식/자료, 문의, 사이트, 근거법령]
        # 정보 수집
        info_dict = dict()
        info_dict['복지 이름'] = driver.find_element_by_tag_name('#contents > div.boardViewHeading.type02 > h4').text
        info_dict['지역구'] = driver.find_element_by_tag_name('#contents > div.boardViewHeading.type02 > div > strong').text
        info_dict['간단 설명'] = driver.find_element_by_tag_name('#contents > div.boardViewHeading.type02 > div > p').text
        
        # 세부사항: 서비스 대상, 서비스 내용, 서비스 이용 및 신청방법, 서식/자료, 문의, 사이트, 근거법령 수집 
        # 복지서비스마다 해당 칼럼 유무 차이있음
        detail_list = driver.find_elements_by_tag_name('#contents > div:nth-child(6) > div.boardViewContent > p')

        # 각 항목에 대한 내용의 태그가 불규칙적으로 소프트코딩하기 위해 일단 세부항목 전체를 가져옴
        detail_text = driver.find_element_by_tag_name('#contents > div:nth-child(6) > div.boardViewContent').text


        for idx in range(len(detail_list)):
            detail_list[idx] = detail_list[idx].text
            print(detail_list[idx])
                         
        for idx in range(len(detail_list)):
            try:
                detail = detail_text.split(detail_list[idx])[1].split(detail_list[idx+1])[0]
            except:
                detail = detail_text.split(detail_list[idx])[1]
            print(detail_list[idx])
            print(detail)
            info_dict[detail_list[idx]] = detail
        
        driver.back()
        time.sleep(7)
        total_info_list.append(info_dict)
        print('==============================')
    driver.close()
        
    return total_info_list


In [61]:
def main():
    current_path = os.getcwd()
    fileName = current_path + '\\복지로3_지자체별\\' + '복지로3_지자체별_복지리스트'
    
    # 폴더 생성
    try:
        if not(os.path.isdir('복지로3_지자체별')):
            os.makedirs(os.path.join('복지로3_지자체별'))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise

    
    url = 'http://www.bokjiro.go.kr/welInfo/retrieveLcgWelInfoList.do?searchSidoCode=&searchCggCode=&searchIntClId=&searchCnDivCd=02&searchCtgId=&searchGb=01&searchText=&pageUnit=10&pageIndex=1'
    #     중간에 끊기는 문제 발생 -> 시작페이지, 끝페이지 쪼개서 작업해야 함.
    #     total_page = get_total_page(url)
    
    # total_page = 50으로 테스트
    # total_page = 50

    start_page = input('시작 페이지 입력')
    end_page = input('끝 페이지 입력')
               
    # 수집 칼럼
    columns = ['복지 이름', '지역구', '간단 설명', '서비스 대상', '서비스 내용', '서비스 이용 및 신청방법', '서식/자료', '문의', '사이트', '근거법령']
    csv_file = open(str(fileName) + '_' + start_page + '_to_' + end_page +'.csv', "w", encoding = 'utf-8-sig', newline = '')
    wr = csv.writer(csv_file)
    wr.writerow(columns)
    
    try:
        # '페이지 이동'에 대한 반복문
        for page in tqdm_notebook(range(int(start_page), int(end_page)+1)):
            url = 'http://www.bokjiro.go.kr/welInfo/retrieveLcgWelInfoList.do?searchSidoCode=&searchCggCode=&searchIntClId=&searchCnDivCd=02&searchCtgId=&searchGb=01&searchText=&pageUnit=10&pageIndex=%s'%(str(page))
            print(page)
            print(url)
            total_info_list = clickNcrawl(url)

            # '모든 수집 복지서비스'에 대한 반복문 -> 복지 서비스 하나
            for info_dict in total_info_list:
                # 'columns'을 돌면서 info_dict가 해당 col을 가지고 있으면 정보를 적고, 없으면 NULL
                wr.writerow([info_dict.get(col, 'NULL') for col in columns])
            print('====================', page, '완료=======================')
        csv_file.close()
        print('저장 완료')
    
    except Exception as e:
        csv_file.close()
        print(page)
        print(e)



## 주의사항: 파싱은 제대로 됨 -> 로딩시간때문에 select가 제대로 안먹기도 함

In [62]:
if __name__ == "__main__":
    main()

시작 페이지 입력136
끝 페이지 입력200


136
http://www.bokjiro.go.kr/welInfo/retrieveLcgWelInfoList.do?searchSidoCode=&searchCggCode=&searchIntClId=&searchCnDivCd=02&searchCtgId=&searchGb=01&searchText=&pageUnit=10&pageIndex=136
서비스 대상
서비스 내용
서비스 이용 및 신청방법
서식/자료
문의
근거법령
서비스 대상

08.7.1 노인장기요양보험법 시행 이전부터 법인 요양시설에 입소한 등급외자


서비스 내용

노인요양시설 입소비용 지원


서비스 이용 및 신청방법

입소한 법인 요양시설에서 관할 시군으로 신청


서식/자료


문의

경기도청 노인복지과 ☎ 031-8008-2563

근거법령

노인복지법 제42조


KeyboardInterrupt: 

In [15]:
a = "서비스 대상 농촌지역(강화군,옹진군)에 거주하고 실제 영농에 종사하는 여성농업인으로서 만 20세 이상～ 만 75세 미만인 자 서비스 내용보건 복지 문화 분야에 이용하는 행복바우처 카드 지원 서비스 이용 및 신청방법 희망자는 군(읍면)에 신청 서식/자료 문의 농축산유통과 ☎ 032-440-4362 근거법령 여성농어업인 육성법 여성농어업인 육성법제3조"


In [16]:
alist = ['서비스 대상', '서비스 내용', '서비스 이용 및 신청방법', '서식/자료', '문의', '사이트', '근거법령']

In [18]:
for i in range(len(alist)):
    try:
        temp = a.split(alist[i])[1].split(alist[i+1])[0]
        print(alist[i])
        print(temp)
        print('==================')
    except:
        print(i)
        temp = a.split(alist[i])[0]
        print(alist[i])
        print(temp)
        print('*******************')

서비스 대상
 농촌지역(강화군,옹진군)에 거주하고 실제 영농에 종사하는 여성농업인으로서 만 20세 이상～ 만 75세 미만인 자 
서비스 내용
보건 복지 문화 분야에 이용하는 행복바우처 카드 지원 
서비스 이용 및 신청방법
 희망자는 군(읍면)에 신청 
서식/자료
 
문의
 농축산유통과 ☎ 032-440-4362 근거법령 여성농어업인 육성법 여성농어업인 육성법제3조
5
사이트
서비스 대상 농촌지역(강화군,옹진군)에 거주하고 실제 영농에 종사하는 여성농업인으로서 만 20세 이상～ 만 75세 미만인 자 서비스 내용보건 복지 문화 분야에 이용하는 행복바우처 카드 지원 서비스 이용 및 신청방법 희망자는 군(읍면)에 신청 서식/자료 문의 농축산유통과 ☎ 032-440-4362 근거법령 여성농어업인 육성법 여성농어업인 육성법제3조
*******************
6
근거법령
서비스 대상 농촌지역(강화군,옹진군)에 거주하고 실제 영농에 종사하는 여성농업인으로서 만 20세 이상～ 만 75세 미만인 자 서비스 내용보건 복지 문화 분야에 이용하는 행복바우처 카드 지원 서비스 이용 및 신청방법 희망자는 군(읍면)에 신청 서식/자료 문의 농축산유통과 ☎ 032-440-4362 
*******************


In [82]:
from copy import deepcopy 

In [ ]:
    try:
        # 정책 설명해주는 obx
        title_box = driver.find_elements_by_tag_name('#contents > div.resultBox > ul > li > div > dl > dt > a')

        for title in title_box:
            title.click()
            time.sleep(2)
            driver.back()
            time.sleep(2)
    except Exception as e:
#         driver.close()
        print(e)

In [30]:
import csv
d1 = {'a':1, 'b':2, 'c': 3}
d2 = {'d':4, 'f': 6}

columns = ['a', 'b', 'c', 'd', 'e', 'f']

# combine d1 and d2 into data.. there are other ways but this is easy to understand
data = dict(d1)
data.update(d2)

with open('my_data.csv','w') as f:
    w = csv.writer(f)
    w.writerow(columns)
    # make a list of values in the order of columns and write them
    w.writerow([data.get(col, 'NULL') for col in columns])

In [31]:
data = dict(d1)
data.update(d2)

In [3]:
data

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6}

In [4]:
[data.get(col, None) for col in columns]

[1, 2, 3, 4, 5, 6]

In [ ]:
data.get

In [ ]:
a = "서비스 대상 농촌지역(강화군,옹진군)에 거주하고 실제 영농에 종사하는 여성농업인으로서 만 20세 이상～ 만 75세 미만인 자 서비스 내용보건 복지 문화 분야에 이용하는 행복바우처 카드 지원 서비스 이용 및 신청방법 희망자는 군(읍면)에 신청 서식/자료 문의 농축산유통과 ☎ 032-440-4362 근거법령 여성농어업인 육성법 여성농어업인 육성법제3조"

alist = ['서비스 대상', '서비스 내용', '서비스 이용 및 신청방법', '서식/자료', '문의', ' 근거법령']

for i in range(len(alist)):
    try:
        temp = a.split(alist[i])[1].split(alist[i+1])[0]
        print(alist[i])
        print(temp)
        print('==================')
    except:
        print(i)
        temp = a.split(alist[i])[1]
        print(alist[i])
        print(temp)
        print('*******************')